# Charity Commission - Register of Charities

In [92]:
from pipeline_utils.db import connect
import pandas as pd
from pathlib import Path

In [93]:
db = connect(read_only=True)

In [94]:
db.query(
'''
CREATE OR REPLACE TEMP TABLE onspd AS SELECT * FROM read_csv('../data/reference/onspd_extract.csv') WHERE oslaua == 'E08000021';
''')

In [95]:
data = db.query('''
    SELECT
        charity_name,
        pcds as charity_postcode,
        organisation_number,
        registered_charity_number,
        oslaua,
        lat as latitude,
        long as longitude
    FROM Charities LEFT JOIN onspd ON charity_contact_postcode == pcds;
    -- SELECT "Charity Name", 'Charity Postcode', 'Charity Type', 'How the charity helps', 'What the charity does', 'Who the charity helps', 'Activities' FROM Charities;
''').df()

In [96]:
db.close()

In [97]:
OUT_DIR = Path('../src/data/charity-commission/_data/')

### Create headline stats

In [98]:
headlines = pd.DataFrame(columns=['headline', 'count'])
headlines['headline'] = ['Total']
headlines['count'] = len(data)

In [99]:
headlines.to_csv(OUT_DIR / 'headlines.csv')

### Visualise by location

In [100]:
data

,charity_name,charity_postcode,organisation_number,registered_charity_number,oslaua,latitude,longitude
0,THE LITERARY AND PHILOSOPHICAL SOCIETY OF NEWC...,NE1 1SE,4034478,1120948,E08000021,54.969197,-1.613461
1,NEWCASTLE ARTS CENTRE TRUST LTD,NE1 1SG,701598,701598,E08000021,54.970002,-1.617999
2,NORTHERN PRINT STUDIO LIMITED,NE1 2NP,5031239,1149872,E08000021,54.974867,-1.594657
3,OUSEBURN TRUST,NE1 2PQ,3950929,1081657,E08000021,54.974814,-1.592361
4,CHILLI STUDIOS (NEWCASTLE AND GATESHEAD ARTS S...,NE1 2TQ,4020661,1116957,E08000021,54.974455,-1.597163
...,...,...,...,...,...,...,...
236,JAZZ NORTH-EAST LIMITED,None,507509,507509,None,NaN,NaN
237,TYNEDALE MUSIC FESTIVAL,None,512652,512652,None,NaN,NaN
238,TETE A TETE PRODUCTIONS LIMITED,None,3949352,1069055,None,NaN,NaN
239,PRUDHOE COMMUNITY BAND,None,3970222,1084048,None,NaN,NaN


In [102]:
geo_data = data.loc[
    data.latitude.notna(),
    ['charity_name', 'charity_postcode', 'latitude', 'longitude']
].dropna().sort_values('charity_name')
geo_data['tooltip'] = geo_data.apply(lambda rec: f"<strong>{rec['charity_name'].lower().title()}</strong><br>Postcode: {rec['charity_postcode']}", axis=1)
geo_data.to_csv(os.path.join(OUT_DIR, 'charity_geo.csv'), index=False)
